In [1]:
import pandas as pd
import numpy as np
import os
os.chdir('../../../')
print(os.getcwd())
from v2.model import Trading
import plotly.graph_objs as go
from plotly.offline import plot
from scipy.signal import argrelextrema

/Users/rosscopeland/Desktop/personal/code/vivaldi/back_testing


In [2]:
def load_config():
    my_config = {}
    with open('config.config') as config:
        for line in config:
            args = line.split('=')
            my_config[args[0]] = args[1].rstrip().split(',')
    return my_config

model = Trading(load_config())

In [3]:
#this only works on one dataset at a time
candle, buys, sells = None, None, None
datasets = model.dfs
fee = 0.0026
for d in datasets:
    buy_prices = []
    buy_times = []
    sell_prices = []
    sell_times = []
    name = d[1]
    cur_min = 99999999999999999.00
    cur_max = 0.00
    cur_min_time = 00.0
    cur_max_time = 00.0
    last_buy_price = 000.00
    last_buy_time = 000.00
    sell_time = 0000.0
    dataset = d[0]
    looking_for_buy = True
    for row in dataset.itertuples():
        close = row.close
        time = row.time
        if looking_for_buy:
            if close < cur_min:
                cur_min = close
                cur_min_time = time
                cur_max = close
                cur_max_time = time
            elif close > cur_max:
                cur_max = close
                cur_max_time = time
                delta = cur_max - cur_min
                if delta > ((cur_min * fee) + (cur_max * fee)):
                    looking_for_buy = False
                    last_buy_price = cur_min
                    last_buy_time = cur_min_time
        else:
            trade_fee = ((cur_min * fee) + (cur_max * fee))
            if close > cur_max:
                cur_max = close
                cur_max_time = time
            elif close < cur_max - trade_fee:
                looking_for_buy = True
                buy_prices.append(cur_min)
                buy_times.append(cur_min_time)
                sell_prices.append(cur_max)
                sell_times.append(cur_max_time)
                cur_min = close
                cur_min_time = time
                cur_max = close
                cur_max_time = time

    candle = go.Candlestick(x=dataset['time'], open=dataset['open'], close=dataset['close'], high=dataset['high'], low=dataset['low'], name='Candlesticks')

    buys = go.Scatter(x=buy_times, y=buy_prices, name='Optimal Entries', mode='markers')
    sells = go.Scatter(x=sell_times, y=sell_prices, name='Optimal Exits', mode='markers')
    dataset['trough'] = dataset.iloc[argrelextrema(dataset.close.values, np.less_equal, order=480)[0]]['close']
    dataset['trough'] = dataset['trough'].fillna(0).gt(0).astype(int)
    troughs = go.Scatter(x=dataset['time'], y=dataset['trough'], name='Troughs (Local Maximums)', mode='markers')
    data = [buys] + [sells] + [candle] + [troughs]
    layout = go.Layout(title='Optimal Buying Times/Detected Troughs for ' + name)
    fig = go.Figure(data=data, layout=layout)
    plot(fig, filename='plots/optimal_' + name + '.html')

In [4]:
sell_prices

[]

In [ ]:
len(buy_price)